In [134]:
import pandas as pd

# خواندن فایل و چک کردن
df = pd.read_csv(r"D:\Laya\DataScience\پارت\fe3abbf0-181d-4621-af4a-0428ff2a53fe_crm part\crm part\data.csv")

print("Unique CustomerIDs:", df["CustomerID"].nunique())
print("Total Rows:", len(df))
print("Nulls in CustomerID:", df["CustomerID"].isna().sum())
print("Nulls in LeadDate:", df["LeadDate"].isna().sum())
print("Date Range:", df["LeadDate"].min(), "to", df["LeadDate"].max())

Unique CustomerIDs: 1000
Total Rows: 1000
Nulls in CustomerID: 0
Nulls in LeadDate: 0
Date Range: 2023-01-01 00:00:00 to 2023-02-11 15:00:00


In [138]:
#  تبدیل تاریخ به فرمت استاندارد

df["LeadDate"] = pd.to_datetime(df["LeadDate"], errors="coerce")

#  حذف فاصله‌های اضافی از متن‌ها
text_columns = ["LeadSource", "Status", "Message", "Sentiment", "Product"]
for col in text_columns:
    df[col] = df[col].astype(str).str.strip()

#  ساخت ستون‌های محاسباتی
df["is_conversion"] = df["Status"].apply(lambda x: 1 if x == "خرید" else 0)
df["is_churn"] = df["Status"].apply(lambda x: 1 if x == "لغو" else 0)

# 5. حذف رکوردهای تاریخ یا آی‌دی نامعتبر (در صورت وجود)
df = df.dropna(subset=["CustomerID", "LeadDate"])

# ذخیره فایل تمیز
df.to_csv("cleaned_data.csv", index=False, encoding="utf-8-sig")

print("✅ فایل cleaned_data.csv ")


✅ فایل cleaned_data.csv 


In [140]:
# سه دلیل نارضایتی مشتری ها

from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
import re

# فیلتر مشتری‌های نارضایتی
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter

# فیلتر مشتریان ناراضی
negative_customers = df[(df['SatisfactionScore'] <= 3) | (df['Sentiment'] == 'منفی')]

# پیش‌پردازش متن‌ها
def preprocess(text):
    text = re.sub(r'[^\w\s]', '', str(text))  # حذف علائم نگارشی
    return text

negative_customers.loc[:, 'clean_message'] = negative_customers['Message'].apply(preprocess)

# استخراج کلیدواژه‌ها
vectorizer = CountVectorizer(stop_words=['و', 'با','شد', 'بعضی','بشه','زیادی','باعث','باید','که','بود','در','تیم','اما','ولی','این','حال', 'شده','برای', 'از','بهتر', 'تا','میکنه','میرسه','نسبت','به','چند','هنوز', 'حرفهای','برامون', 'خیلی', 'همین'])
X = vectorizer.fit_transform(negative_customers['clean_message'])
keywords = vectorizer.get_feature_names_out()

# شمارش تکرار کلیدواژه‌ها
word_counts = X.toarray().sum(axis=0)
word_freq = dict(zip(keywords, word_counts))

# کلیدواژه‌های پرتکرار (مثلا 20 تا)
top_words = Counter(word_freq).most_common(20)

# ذخیره در DataFrame و خروجی CSV
word_df = pd.DataFrame(top_words, columns=["Word", "Frequency"])
word_df.to_csv("negative_keywords.csv", index=False, encoding="utf-8-sig")

print("✅ فایل negative_keywords.csv ساخته شد.")
display(word_df.head())


✅ فایل negative_keywords.csv ساخته شد.


C:\Users\LNm\AppData\Local\Temp\ipykernel_2028\223100913.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  negative_customers.loc[:, 'clean_message'] = negative_customers['Message'].apply(preprocess)


,Word,Frequency
0,پشتیبانی,221
1,کیفیت,140
2,اشتراک,132
3,ماژول,131
4,نرمافزار,126


In [142]:
# فیلتر پیام‌های منفی (رضایت پایین + Sentiment منفی)
df_neg = df[(df["SatisfactionScore"] <= 2) & (df["Sentiment"] == "منفی")]

# شمارش پیام‌های منفی برای هر محصول
product_issues = df_neg["Product"].value_counts().reset_index()
product_issues.columns = ["Product", "NegativeMessages"]

# ذخیره به CSV
product_issues.to_csv("negative_messages_by_product.csv", index=False)

# نمایش
display(product_issues)


,Product,NegativeMessages
0,ماژول مدیریت تیکت پشتیبانی,20
1,اشتراک CRM شش‌ماهه,20
2,ماژول گزارش‌گیری پیشرفته,19
3,اشتراک CRM یک‌ماهه,16
4,پکیج کامل CRM (فروش+بازاریابی+پشتیبانی),14
5,ماژول اتوماسیون بازاریابی,14
6,اشتراک CRM یک‌ساله,12


In [114]:
# شمارش پیام‌های منفی برای هر منبع جذب
leadsource_issues = df_neg["LeadSource"].value_counts().reset_index()
leadsource_issues.columns = ["LeadSource", "NegativeMessages"]

# ذخیره به CSV
leadsource_issues.to_csv("negative_messages_by_leadsource.csv", index=False)

# نمایش
display(leadsource_issues)


,LeadSource,NegativeMessages
0,ارجاع مشتری,28
1,تبلیغ گوگل,28
2,نمایشگاه,22
3,اینستاگرام,20
4,وب‌سایت,17
